In [2]:
from bioblocks.io import read_model, write_model
from bioblocks.transform import rename_chains
from anarci import anarci
from Bio import pairwise2
from Bio.pairwise2 import format_alignment
from Bio.PDB import PDBList, PDBParser, PDBIO, Select
from Bio.PDB import MMCIFParser, PDBIO, Select
import requests
import os


/home/yuyang/.conda/envs/ppi/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [3]:

def fetch_pdb_biounit(pdb_id, assembly_id=1, output_file=None):

    pdb_id = pdb_id.lower()
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb{assembly_id}"
    if output_file is None:
        output_file = f"{pdb_id}_pdb{assembly_id}.pdb"

    print(f"⬇️  Downloading biological assembly {assembly_id} for {pdb_id} ...")
    r = requests.get(url)

    if r.status_code != 200:
        raise ValueError(f"❌ 无法下载 {pdb_id} assembly {assembly_id} (HTTP {r.status_code})")

    with open(output_file, "wb") as f:
        f.write(r.content)

    print(f"✅ Saved -> {output_file}")
    return output_file

In [4]:
def classify_chain(seq, chain_id=None, scheme="chothia", ncpu=1):
    
    name = chain_id if chain_id else "seq"
    alignment_results, hit_tables, domain_alignments = anarci([(name, seq)], 
                                                              scheme=scheme, 
                                                              ncpu=ncpu, 
                                                              output=False)

    if not hit_tables or hit_tables[0] is None or len(hit_tables[0]) == 0:
        return "None"

    chain_type = hit_tables[0][0]["chain_type"]
    if chain_type == "H":
        return "H"
    elif chain_type in ["K", "L"]:
        return "L"
    else:
        return "A"

In [5]:
def sequence_similarity(seq1, seq2, ):

    seq1, seq2 = seq1.strip().upper(), seq2.strip().upper()

    alignments = pairwise2.align.globalxx(seq1, seq2)

    best_alignment = alignments[0]
    aln_seq1, aln_seq2, score, start, end = best_alignment

    matches = sum(a == b for a, b in zip(aln_seq1, aln_seq2))
    identity = matches / max(len(seq1), len(seq2))

    similarity = identity

    return {
        "identity": identity,
        "similarity": similarity,
        "alignment": format_alignment(*best_alignment)
    }

In [14]:
PDB_NAME = "5njd.pdb"
PDB_DIR = "/home/yuyang/lb/data/IL23"
PDB_FILE = os.path.join(PDB_DIR, PDB_NAME)
PDB_OUTPUT_DIR = PDB_DIR
AF3_example_cif = f"{PDB_DIR}/cif/MJ00D_0.cif"

fetch_pdb_biounit("5njd", output_file=PDB_FILE)

model = read_model(PDB_FILE)
af3_model = read_model(AF3_example_cif)

⬇️  Downloading biological assembly 1 for 5njd ...
✅ Saved -> /home/yuyang/lb/data/IL23/5njd.pdb


In [15]:
af3_model["A"].sequence

'MCHQQLVISWFSLVFLASPLVAIWELKKDVYVVELDWYPDAPGEMVVLTCDTPEEDGITWTLDQSSEVLGSGKTLTIQVKEFGDAGQYTCHKGGEVLSHSLLLLHKKEDGIWSTDILKDQKEPKNKTFLRCEAKNYSGRFTCWWLTTISTDLTFSVKSSRGSSDPQGVTCGAATLSAERVRGDNKEYEYSVECQEDSACPAAEESLPIEVMVDAVHKLKYENYTSSFFIRDIIKPDPPKNLQLKPLKNSRQVEVSWEYPDTWSTPHSYFSLTFCVQVQGKSKREKKDRVFTDKTSATVICRKNASISVRAQDRYYSSSWSEWASVPCS'

In [23]:
mapping_dict = {}
for chain in model.get_chains():
    seq = model[chain.id].sequence
    classification = classify_chain(seq, scheme="kabat")
    mapping_dict[chain.id] = classification

In [24]:
antigen_chain_ids = [k for k, v in mapping_dict.items() if v == "None"]

In [25]:
mapping_dict

{'A': 'None', 'B': 'None', 'U': 'L', 'V': 'H', 'Y': 'None'}

In [27]:
target_antigen_seq = af3_model["A"].sequence
for antigen_chain_id in antigen_chain_ids:
    potential_seq = model[antigen_chain_id].sequence
    result = sequence_similarity(target_antigen_seq, potential_seq)
    if result["identity"] > 0.9:
        mapping_dict[antigen_chain_id] = "A"
    else:
        del mapping_dict[antigen_chain_id]

In [31]:
mapping_dict

{'A': 'A', 'U': 'L', 'V': 'H'}

In [29]:
new_model = rename_chains(model=model, rename_mapping=mapping_dict)

In [30]:
write_model(new_model, f"{PDB_FILE}")

In [37]:
from bioblocks.io import read_model, write_model
from bioblocks.transform import filter_residues
from anarci import anarci
from Bio.Align import PairwiseAligner
import Bio
import os
import pandas as pd
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

import sys

def extract_variable_region_sequence(seq: str) -> str:
    """Helper function to extract the variable region sequence of a single chain using ANARCI.

    Args:
        seq: The full VH VL sequence

    Returns:
        The sequence after alignment to the Chothia numbering scheme (including gaps).
    """
    result, _, _ = anarci([(0, seq)], ncpu=1, scheme="chothia", output=False)  # type: ignore[no-untyped-call]
    return "".join([t[1] for t in result[0][0][0]])

def get_continuous_sequence_start_end_position(
    query_sequence: str,
    target_sequence: str,
    align_if_substring: bool = False,  # noqa: FBT002, FBT001
) -> tuple[int, int, Bio.Align.PairwiseAlignments]:
    """Get the start and end positions of the query sequence in the reference sequence.

    The query sequence may carry mutations.

    Args:
        query_sequence: Query sequence.
        target_sequence: Reference sequence which anchors the query sequence.
        align_if_substring: Whether to align the query sequence if it is a substring of the target sequence.

    Returns:
        - Start position of the query sequence in the target sequence, the numbering starts at 1.
        - End position of the query sequence in the target sequence (inclusive)
        - Pairwise alignment object if the query sequence is not found in the target sequence.
    """
    idx_start = target_sequence.find(query_sequence)
    idx_end = idx_start + len(query_sequence)
    if (not align_if_substring) and (idx_start != -1):
        return idx_start + 1, idx_end, None

    aligner = PairwiseAligner(scoring="blastp")
    # query is linear, open/extend gap on the ends are fine
    aligner.query_left_gap_score = 0.0
    aligner.query_left_extend_gap_score = 0.0
    aligner.query_right_gap_score = 0.0
    aligner.query_right_extend_gap_score = 0.0
    alignment = aligner.align(target_sequence, query_sequence)[0]
    idx_start, idx_end = alignment.aligned[0][0][0], alignment.aligned[0][-1][-1]
    # aligned contains start and end indices of subsequences in the target and query sequence
    # aligned[0] is target and aligned[0][0] is (t_start1, t_end1) the first trunk, aligned[1] is query
    return idx_start + 1, idx_end, alignment


def trim_vlvh(pdb_model, chain_id, ref_seq):
    original_start_index = list(pdb_model[chain_id].get_residues())[0].id.residue_index

    variable_region_seq = extract_variable_region_sequence(ref_seq).replace("-", "X")
    start, end, seq = get_continuous_sequence_start_end_position(variable_region_seq, ref_seq, align_if_substring=True)

    gap_range = original_start_index - start

    final_start_index = gap_range + 1
    final_end_index = end + gap_range

    return final_start_index, final_end_index

def get_ab_residue_filter(h_range, l_range):
    def residue_filter(res):
        chain_id = res.parent.id
        res_id = res.id.residue_index

        if chain_id == "H":
            return res_id in h_range
        elif chain_id == "L":
            return res_id in l_range
        elif chain_id == "A":
            return True
        else:
            return False
    return residue_filter

def trim_vhvl_from_pdb(pdb_file: str, pdb_dir: str, out_dir: str):
    file_name = pdb_file.replace(".pdb", "")
    pdb_model = read_model(os.path.join(pdb_dir, pdb_file))

    chain_id_list = [chain.id for chain in pdb_model.get_chains()]
    h_range = []
    l_range = []
    try:
        if "H" in chain_id_list:
            # extract_vl
            heavy_chain_seq = pdb_model["H"].sequence
            h_start, h_end = trim_vlvh(pdb_model, "H", heavy_chain_seq)
            h_range = range(h_start, h_end + 1)
        if "L" in chain_id_list:
            light_chain_seq = pdb_model["L"].sequence
            l_start, l_end = trim_vlvh(pdb_model, "L", light_chain_seq)
            l_range = range(l_start, l_end + 1)

    except Exception as e:
        print(f"Error processing {pdb_file}: {e}")
        return None
    
    residue_filter = get_ab_residue_filter(h_range, l_range)
    ab1_trimmed = filter_residues(pdb_model, residue_filter=residue_filter)

    vh_seq = ab1_trimmed["H"].sequence if "H" in chain_id_list else ""
    vl_seq = ab1_trimmed["L"].sequence if "L" in chain_id_list else ""
    antigen_seq = pdb_model["A"].sequence

    write_model(ab1_trimmed, os.path.join(out_dir, f"{file_name}.pdb"))
    return file_name, vh_seq, vl_seq, antigen_seq



In [38]:
trim_vhvl_from_pdb(PDB_NAME, PDB_DIR, PDB_OUTPUT_DIR)

('5njd',
 'QVQLVESGGGVVQPGRSLRLSCAASGFTFSSYGMHWVRQAPGKGLEWVAFIRYDGSNKYYADSVKGRFTISRDNSKNTLYLQMNSLRAEDTAVYYCKTHGSHDNWGQGTMVTVSS',
 'QSVLTQPPSVSGAPGQRVTISCSGSRSNIGSNTVKWYQQLPGTAPKLLIYYNDQRPSGVPDRFSGSKSGTSASLAITGLQAEDEADYYCQSYDRYTHPALLFGTGTKVTVL',
 'IWELKKDVYVVELDWYPDAPGEMVVLTCDTPEEDGITWTLDQSSEVLGSGKTLTIQVKEFGDAGQYTCHKGGEVLSHSLLLLHKKEDGIWSTDILKDQKEPKNKTFLRCEAKNYSGRFTCWWLTTISTDLTFSVKSSRGSSDPQGVTCGAATLSAERVRGDNKEYEYSVECQEDSACPAAEESLPIEVMVDAVHKLKYENYTSSFFIRDIIKPDPPKNLQLKPLKQVEVSWEYPDTWSTPHSYFSLTFCVQVQEKKDRVFTDKTSATVICRKNASISVRAQDRYYSSSWSEWASVPCS')